In [ ]:
import numpy as np
from scipy.optimize import linprog

In [ ]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        if i==0:
          arches[k]="s->"+str(j+1)
        elif j==0:
          arches[k]=str(i+1)+"->s"
        elif i==columns-1:
          arches[k]="t->"+str(j+1)
        elif j==columns-1:
          arches[k]=str(i+1)+"->t"
        else:
          arches[k]=str(i+1)+"->"+str(j+1)
        k+=1
  return NA, arches


def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

In [ ]:

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])


Aeq, arc_idxs = NN2NA(NN)
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])
beq = np.array([0, 0, 0, 0, 0, 0])
max_q = [7, 1, 2, 3, 2, 1, 2, None]
bounds = tuple([(0, max_q[arcs]) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n\n '
      'Arches in order:\n%s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

## Optimizer inputs ## 
Cost vector: [ 0  0  0  0  0  0  0 -1] 

 Arches in order:
['s->2', 's->3', '2->4', '2->t', '3->5', '4->t', '5->t', '6->s'] 

A_eq Node-Arc matrix:
[[ 1.  1.  0.  0.  0.  0.  0. -1.]
 [-1.  0.  1.  1.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.  0.]
 [ 0.  0.  0. -1.  0. -1. -1.  1.]] 

b_eq demand-supply vector: [0 0 0 0 0 0] 

Bounds of each X arc variable: ((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None))


In [ ]:
# OPTIMIZE:

res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

# GET THE SOLUTION:
usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print('## Results ##')
print('The usage of each arc will be %s' % usage)
print('The arcs that produce the minimum cut : %s' % min_cut)
print('The maximum flow will be: %0.2f\n\n\n' % max_flow)

## Results ##
The usage of each arc will be (from, to) {'s->2': '4/7', 's->3': '1/1', '2->4': '1/2', '2->t': '3/3', '3->5': '1/2', '4->t': '1/1', '5->t': '1/2', '6->s': '5/None'}
The arcs that produce the minimum cut (from, to): ['s->3', '2->t', '4->t']
The maximum flow will be: 5.00





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  This is separate from the ipykernel package so we can avoid doing imports until
